In [50]:
import os
import re
import logging
from pathlib import Path

#TODO: Move all to pathlib

# Where the files come from
src_root_folder = "C:\\Users\\Kasper\\OneDrive\\Bujinkan"
# Where the files go
dest_root_folder = "C:\\Users\\Kasper\\Desktop\\Converted-lib2"

# Global logger
log_path = Path(os.path.join(src_root_folder, "convertion.log"))
logger = logging.getLogger(__name__)
logging.basicConfig(filename=log_path, level=logging.INFO)

In [51]:
from file_read_backwards import FileReadBackwards


def purge_unfinished_file_from_logger() -> None:
    if not log_path.is_file() or os.stat(log_path).st_size == 0:
        return

    with FileReadBackwards(
        log_path, encoding="utf-8"
    ) as log_file:  # Read file from the back

        while True:
            line = str(log_file.readline())

            if line == "" or line == None:  # Skip end of or empty file
                continue
            elif (
                re.search("Finished", line) != None
            ):  # The last item finished. nothing to see here
                print('No unfinished file to remove from last run')
                return
            else:
                last_download = re.search(
                    "INFO:.*?:Started - (.*)", line
                )  # Look for 'download started' in line

                if last_download != None:  # Save and finish
                    unfinished_file = f'{last_download[1].strip("\r\n").strip('\r').strip('\n').strip()}.mp4'

                    if os.path.isfile(os.path.join(dest_root_folder, unfinished_file)):
                        os.remove(os.path.join(dest_root_folder, unfinished_file))
                        print(
                            f"Removed unfinished file from last download: '{unfinished_file}'\n"
                        )

                    return

In [52]:
import ffmpeg

# TODO: Figure out nvenc
# TODO: Consider av1
# FIXME: Crashes on big batches (with many failing videos)
def convert_stream_to_hvec(
    path_and_name: str,
    input_format: str,
    delete_source: bool = False
):
    logger.info(f"Started - {path_and_name}")

    src_file = os.path.join(src_root_folder, f"{path_and_name}.{input_format}")
    dest_file = os.path.join(dest_root_folder, f"{path_and_name}.mp4")

    print(f"Starting convertion of {src_file} to {dest_file}")

    try:
        ffmpeg.input(src_file, protocol_whitelist="https,file,tls,tcp").output(

            dest_file, vcodec="libx265", vtag="hvc1"
        ).run()  # TODO: Progress counter

        # Set to finished so file isn't removed next run (overwrite == False)
        logger.info("Finished")
        
        # Safe to remove old file
        if delete_source:
            os.remove(src_file)

        return True

    except Exception:
        print(f"Convertion failed. Moving on.")

        if os.path.exists(dest_file):
            os.remove(dest_file)

        return False

    finally:
        # Close
        logger.info("Finished")

In [53]:
def print_m3u8_files():
    for folder in os.walk(src_root_folder):
        folder_path = folder[0]

        for file in folder[2]:
            if not file.endswith(".m3u8"):
                continue

            file_name = re.search("(.*).m3u8$", file)[1]
            path_and_name = os.path.join(folder_path, file_name)

            print(path_and_name)

In [54]:
def convert_m3u8(overwrite: bool = True):
    # Remove file that didn't finish downloading in log-file
    purge_unfinished_file_from_logger()

    files_to_convert: list[str] = []
    # TODO: Print files removed/ignored

    # Recursive find all in directory
    for folder in os.walk(src_root_folder):
        folder_path = folder[0]

        for file in folder[2]:
            if not file.endswith(".m3u8"):
                continue

            file_name = re.search("(.*).m3u8$", file)[1]
            path_and_name = os.path.join(folder_path, file_name)

            # The file already exists
            if os.path.exists(f"{path_and_name}.mp4"):
                # Should be overwritten
                if overwrite:
                    print(f"Overwriting old file: {path_and_name}")
                    os.remove(f"{path_and_name}.mp4")
                # Ignore new download request
                else:
                    print(f"File already exists: {path_and_name}")
                    os.remove(f"{path_and_name}.m3u8")
                    continue

            files_to_convert.append(path_and_name)

    print(f"Found {len(files_to_convert)} files to convert")

    failed_convertions: int = 0

    for index, file in enumerate(files_to_convert):
        if convert_stream_to_hvec(file, 'm3u8', True):
            print(f"Converted {index + 1} of {len(files_to_convert)} files")
        else :
            failed_convertions += 1
            print (f"Failed to convert file {index + 1} of {len(files_to_convert)}")
        
    print(f"Finished {len(files_to_convert) - failed_convertions} of {len(files_to_convert)}{f". {failed_convertions} failed" if failed_convertions > 0 else ""}")

In [55]:
from file_organizers import create_directory

def convert_local_media(overwrite: bool = False):
    # Remove file that didn't finish downloading in log-file
    purge_unfinished_file_from_logger()

    # path, file-name, extension
    files_to_convert: list[(str, str, str)] = []
    # TODO: Print files removed/ignored

    # Recursive find all in directory
    for folder in os.walk(src_root_folder):
        #TODO: smarter prefix removal
        folder_path = folder[0].removeprefix(src_root_folder).removeprefix('\\').removeprefix('/')        

        for file in folder[2]:
            if not file.lower().endswith((".avi", '.vob', '.mpg', 'mp4', '.wmv')):
                continue

            file_parts = re.search(r"(.*)\.(.*)$", file) 
            file_name = file_parts[1]
            file_type = file_parts[2]
            dist_path_and_name = os.path.join(dest_root_folder, folder_path, file_name)


            # The file already exists
            if os.path.exists(f"{dist_path_and_name}.mp4"):
                # Should be overwritten
                if overwrite:
                    print(f"Overwriting old file: {dist_path_and_name}")
                    os.remove(f"{dist_path_and_name}.mp4")
                # Ignore new convertion request
                else:
                    print(f"File already exists: {dist_path_and_name}")
                    continue

            files_to_convert.append((folder_path, file_name, file_type))

    print(f"Found {len(files_to_convert)} files to convert")

    failed_convertions: int = 0

    for index, file in enumerate(files_to_convert):
        dest_path = os.path.join(dest_root_folder, file[0])

        if(not Path(dest_path).exists()):
            create_directory(dest_path)

        if convert_stream_to_hvec(os.path.join(file[0], file[1]), file[2] ):
            print(f"Converted {index + 1} of {len(files_to_convert)} files")
        else :
            failed_convertions += 1
            print (f"Failed to convert file {index + 1} of {len(files_to_convert)}")
        
    print(f"Finished {len(files_to_convert) - failed_convertions} of {len(files_to_convert)}{f". {failed_convertions} failed" if failed_convertions > 0 else ""}")

In [56]:
convert_local_media()

print("Done")

No unfinished file to remove from last run

File already exists: C:\Users\Kasper\Desktop\Converted-lib2\01 - Naginata, Nagamaki & Bisento
File already exists: C:\Users\Kasper\Desktop\Converted-lib2\02 - Ken,Tachi And Katana
File already exists: C:\Users\Kasper\Desktop\Converted-lib2\03 - Rokushaku Bo Jutsu
File already exists: C:\Users\Kasper\Desktop\Converted-lib2\04 - kobudo no kihon
File already exists: C:\Users\Kasper\Desktop\Converted-lib2\05 - What Is Martial Arts
File already exists: C:\Users\Kasper\Desktop\Converted-lib2\06 - Sojutsu Yari
File already exists: C:\Users\Kasper\Desktop\Converted-lib2\08 - Takagi yoshin Ryu - Mutodori
File already exists: C:\Users\Kasper\Desktop\Converted-lib2\09 - Hidden Weapons Shuriken Techniques
File already exists: C:\Users\Kasper\Desktop\Converted-lib2\10 - Budo Of Elegance
File already exists: C:\Users\Kasper\Desktop\Converted-lib2\11 - Shinkenshirahdome
File already exists: C:\Users\Kasper\Desktop\Converted-lib2\12 - Shindenfudo Ryu Happo B